In [1]:
"""
Avi Shah, 11/14/2022. Yang Lab NJMS

This script traverses the direct download
from Illumina's basespace and generates scripts
that can be run to align reads to a given reference
genome.
"""

import os
import pandas as pd
import numpy as np
import scipy

# we need to get to scratch space; we start on jupyter
# all of the files from Illumina's basespace were
# downloaded to /scratch/as2654/RNAseq0 using
# the basespace CLI on Amarel
scrpath = os.path.join("/scratch", "as2654", "RNAseq0")
print(scrpath)
os.chdir(scrpath)

# OS walk the download folder as root to ensure
# that all gzipped sequencing files are present;
# also have indices/annot for tuberculosis and coli.
# NOTE: indices exported to BOWTIE2_INDEXES global var
for root, subdirs, files in os.walk("."):
    print(root)
    print("\n", len(subdirs))
    for sd in subdirs:
        print(sd)
    print()
    for f in files:
        print(f)


/scratch/as2654/RNAseq0
.

 86
11_L002_ds.9f49b088c9f74001b79064567d253a75
10_L002_ds.a94c1831f6d444e1855bbaad85082f0d
3_L002_ds.ba6aafd4e2ba43c485c280fb4ec5e79e
34_L001_ds.bf7127527551495fa1a4a58ca519903b
23_L001_ds.a33e5b8d9f7a4ed8bc31553c2c2da2d7
7_L001_ds.51ddfbd5a90f471eb55080bfc0aea372
18_L002_ds.350e25fc4a7e484293ae4b9a9767dcaa
12_L001_ds.7f8e264de73747d3894f21f9e8788339
25_L002_ds.929b45e3d4264bd49781dac51853d598
1_L001_ds.1969ed4d36b04971be773b320148a85b
14_L002_ds.322d5950daa448f088d0bbf4fa53f7a4
24_L002_ds.66df139c925b4bb382f335ed03d37bc6
3_L001_ds.0c295149fece4fddbe13631e39190018
8_L002_ds.1dd835d2dd6c44dba2dd59d512876bb2
33_L001_ds.38f7687b4bc046dda7c91dd3e3e6f056
16_L001_ds.41803543209b41f58820a74b825b7922
25_L001_ds.4de6e9a3462945baaaf13467a366303b
8_L001_ds.5863f1a569bc4a889c66aca8753b2ca9
17_L002_ds.1ea009361d5b41229933ceaef034f26a
18_L001_ds.3def297ea83a453cbb30684d6b865bc8
22_L002_ds.5d9fd11530f94f9e9979d7fb10f445df
23_L002_ds.df591ff5fa944bd7858188c9076147f3
16_L002

In [2]:
# get the index of files provided by genomics crew

import pandas as pd

# this is the first sheet from:
# "Illumina Library Submission_Yang_101222_Index.xlsx"
# saved as csv
indices = pd.read_csv("index.csv")

for row, data in indices.items():
    print(row)
    for key, value in data.items():
        print(key, value)

Well
0 A1
1 B1
2 C1
3 D1
4 E1
5 F1
6 G1
7 H1
8 A2
9 B2
10 C2
11 D2
12 E2
13 F2
14 G2
15 H2
16 A3
17 B3
18 C3
19 D3
20 E3
21 F3
22 G3
23 H3
24 A4
25 B4
26 C4
27 D4
28 E4
29 F4
30 G4
31 H4
32 A5
33 B5
34 C5
35 nan
36 Separate:
37 nan
38 NEB Oligos: https://www.neb.com/-/media/nebus/files/manuals/manuale6609.pdf 
Sample Name
0 CTL1
1 CTL2
2 CTL3
3 I2
4 I3
5 IB1
6 IB2
7 IB3
8 IR1
9 IR2
10 IR3
11 IC1
12 IC2
13 IC3
14 IQ1
15 IQ2
16 IQ3
17 RV1
18 RV2
19 RV3
20 RV4
21 RV5
22 RV6
23 KG1
24 KG2
25 KG3
26 KG4
27 KG5
28 KG6
29 WT1-BL
30 WT2-BL
31 WT3-BL
32 atpA1-BL
33 atpA2-BL
34 atpA3-BL
35 nan
36 I1
37 nan
38 nan
NEB Oligos Well
0 A2
1 B2
2 C2
3 D2
4 E2
5 F2
6 G2
7 H2
8 A3
9 B3
10 C3
11 D3
12 E3
13 F3
14 G3
15 H3
16 A4
17 B4
18 C4
19 D4
20 E4
21 F4
22 G4
23 H4
24 A5
25 B5
26 C5
27 D5
28 E5
29 F5
30 G5
31 H5
32 D1
33 E1
34 F1
35 nan
36 A1
37 nan
38 nan
NEB Oligos Primer
0 P2
1 P14
2 P26
3 P38
4 P50
5 P62
6 P74
7 P86
8 P3
9 P15
10 P27
11 P39
12 P51
13 P63
14 P75
15 P87
16 P4
17 P16
18 P28
19 P40
2

In [3]:
import math
import csv

# cross reference the index and the data downloaded from
# Illumina's basespace to make sure all the files are there
# and enforce that the names should match

# first i will collect sample names

sample_mdmap = dict()

# save all the sample names and the organism they came from
with open("index.csv") as csvf:
    reader = csv.reader(csvf, delimiter=",")
    for i, line in enumerate(reader):
        #print('line[{}] = {}'.format(i, line))
        orgstr = ""
        if line[4].startswith("M. tuberculosis"):
            orgstr = "mtb"            
        elif line[4].startswith("E. coli"):
            orgstr = "ecoli"
        #print(line[1], orgstr)
        if orgstr:
            #print(i)
            sample_mdmap[line[1]] = orgstr

for sample_name, organism in sample_mdmap.items():
    print(sample_name, organism)

print(len(sample_mdmap.items()))




CTL1 mtb
CTL2 mtb
CTL3 mtb
I2 mtb
I3 mtb
IB1 mtb
IB2 mtb
IB3 mtb
IR1 mtb
IR2 mtb
IR3 mtb
IC1 mtb
IC2 mtb
IC3 mtb
IQ1 mtb
IQ2 mtb
IQ3 mtb
RV1 mtb
RV2 mtb
RV3 mtb
RV4 mtb
RV5 mtb
RV6 mtb
KG1 mtb
KG2 mtb
KG3 mtb
KG4 mtb
KG5 mtb
KG6 mtb
WT1-BL ecoli
WT2-BL ecoli
WT3-BL ecoli
atpA1-BL ecoli
atpA2-BL ecoli
atpA3-BL ecoli
I1 mtb
36


In [4]:
# now I will traverse the download and collect names

dl_names = []

# we need to ignore data from the fastq generate that took
# place on 11/7; we want only the data from 11/5

# the files from 11/7 are much smaller; detect by size

# NOTE: met with Dr. Yang 11/14/2022, small files are from
# a failed fastq generation run, do not process

# save the filename pair and their combined size
pairs = []

# save specifically the ones with real data
process_pairs = []

os.chdir(scrpath)
for fn0 in os.listdir("."):
    # generate absolute path for each
    fn0_ap = os.path.join(scrpath, fn0)
    # if it's a subdirectory, it may have my gzipped reads
    if os.path.isdir(fn0_ap):
        print(fn0_ap)
        pair = set(os.listdir(fn0_ap))
        pair_ap = []
        size = 0
        for fn1 in os.listdir(fn0_ap):
            print("   ", os.path.getsize(os.path.join(fn0_ap, fn1)) / 1e6, fn1)
            fn1_ap = os.path.join(fn0_ap, fn1)
            pair_ap.append(fn1_ap)
            size += os.path.getsize(fn1_ap) / 1e6
        pairs.append([pair, size])
        if size > 1200:
            dl_names.append(fn1.split("_")[0])
            process_pairs.append([pair_ap])
        print()

print(len(dl_names))
print(len(set(dl_names)))
        

            

/scratch/as2654/RNAseq0/11_L002_ds.9f49b088c9f74001b79064567d253a75
    842.218075 IR3_S11_L002_R2_001.fastq.gz
    805.421656 IR3_S11_L002_R1_001.fastq.gz

/scratch/as2654/RNAseq0/10_L002_ds.a94c1831f6d444e1855bbaad85082f0d
    817.449538 IR2_S10_L002_R1_001.fastq.gz
    910.150762 IR2_S10_L002_R2_001.fastq.gz

/scratch/as2654/RNAseq0/3_L002_ds.ba6aafd4e2ba43c485c280fb4ec5e79e
    870.013187 CTL3_S3_L002_R1_001.fastq.gz
    919.083091 CTL3_S3_L002_R2_001.fastq.gz

/scratch/as2654/RNAseq0/34_L001_ds.bf7127527551495fa1a4a58ca519903b
    968.534159 atpA2-BL_S34_L001_R1_001.fastq.gz
    1021.909132 atpA2-BL_S34_L001_R2_001.fastq.gz

/scratch/as2654/RNAseq0/23_L001_ds.a33e5b8d9f7a4ed8bc31553c2c2da2d7
    925.52456 RV6_S23_L001_R2_001.fastq.gz
    853.53278 RV6_S23_L001_R1_001.fastq.gz

/scratch/as2654/RNAseq0/7_L001_ds.51ddfbd5a90f471eb55080bfc0aea372
    0.000668 IB2_S7_L001_R2_001.fastq.gz
    0.000641 IB2_S7_L001_R1_001.fastq.gz

/scratch/as2654/RNAseq0/18_L002_ds.350e25fc4a7e484293ae4b

In [5]:
# enforce that all sample names are present in both
# the genomics crew's index and the files that
# Illumina provided via basespace

genomics_names = [k for k, v in sample_mdmap.items()]
dl_names = dl_names
print(genomics_names, dl_names)
print(set(genomics_names) - set(dl_names))
print(set(dl_names) - set(genomics_names))
# these should BOTH print the empty set

['CTL1', 'CTL2', 'CTL3', 'I2', 'I3', 'IB1', 'IB2', 'IB3', 'IR1', 'IR2', 'IR3', 'IC1', 'IC2', 'IC3', 'IQ1', 'IQ2', 'IQ3', 'RV1', 'RV2', 'RV3', 'RV4', 'RV5', 'RV6', 'KG1', 'KG2', 'KG3', 'KG4', 'KG5', 'KG6', 'WT1-BL', 'WT2-BL', 'WT3-BL', 'atpA1-BL', 'atpA2-BL', 'atpA3-BL', 'I1'] ['IR3', 'IR2', 'CTL3', 'atpA2-BL', 'RV6', 'RV1', 'IC1', 'KG2', 'CTL1', 'IC3', 'KG1', 'CTL3', 'IB3', 'IQ2', 'IQ3', 'RV5', 'RV6', 'IQ2', 'IC2', 'atpA3-BL', 'IR1', 'atpA2-BL', 'RV1', 'WT1-BL', 'IR1', 'RV2', 'KG6', 'IB1', 'I2', 'IR2', 'WT3-BL', 'IQ1', 'RV3', 'atpA1-BL', 'KG4', 'KG4', 'WT2-BL', 'IR3', 'IC2', 'WT1-BL', 'IB2', 'CTL1', 'I1', 'KG5', 'I3', 'KG2', 'IB3', 'I1', 'CTL2', 'KG6', 'IC3', 'IQ3', 'atpA3-BL', 'I2', 'KG3', 'WT3-BL', 'I3', 'KG3', 'RV4', 'IQ1', 'KG5', 'WT2-BL', 'CTL2', 'IB2', 'RV2', 'RV3', 'IC1', 'KG1', 'RV4', 'atpA1-BL', 'IB1', 'RV5']
set()
set()


In [8]:
# start building the submission files
# genomics crew demuxed and adapter trimmed
# the fastq files during fastq generation, 
# these steps are not incorporated in the 
# present pipeline. also see https://doi.org/10.1093/nargab/lqaa068

# main parameters for each job
n_cores     = 8
ram         = 16000 #MB
work_dir    = os.path.join("/mnt", "scratch")
save_dir    = os.path.join("/scratch", "as2654", "RNAseq0")
# bowtie2 index prefix, annotations (.gtf), and featureCounts feature
# for M. tubercolosis samples
bt2_mtb     = "H37Rv_Alland"
gtf_mtb     = os.path.join("/scratch", "as2654", "RNAseq0", "H37Rv_Alland.gtf")
fcpfx_mtb   = "transcript"
# for E. coli
bt2_ecoli   = "ecoli"
gtf_ecoli   = os.path.join("/scratch", "as2654", "RNAseq0", "ecoli.gtf")
fcpfx_ecoli = "gene"

n_nodes     = 6 # max number of active nodes
workload    = [[] for k in range(n_nodes)]

def get_filename(abspath):
    return abspath.split("/")[-1]

# make sure all the files are there (i should count to 72)
# and enforce 
i = 0
print(len(process_pairs))
for pair_idx in range(len(process_pairs)):
    pair = process_pairs[pair_idx]
    pair[0].sort(key=get_filename)
    rfor = pair[0][0].split("/")[-1]
    rrev = pair[0][1].split("/")[-1]
    unique_pfxfor = rfor.split("_R1_00")[0]
    unique_pfxrev = rrev.split("_R2_00")[0]
    sample_name = pair[0][0].split("/")[-1].split("_")[0]
    if unique_pfxfor == unique_pfxrev:
        #print(unique_pfxfor)
        i += 1
        # files verified 11/15/2022, 12:12pm
        command_list =  ["./process_reads.sh"] # $1
        command_list += [pair[0][0]]           # $2 etc.
        command_list += [pair[0][1]]
        command_list += [str(n_cores)]
        command_list += [str(ram)]
        command_list += [work_dir]
        command_list += [save_dir]
        if sample_mdmap[sample_name] == "mtb":
            command_list += [bt2_mtb]
            command_list += [gtf_mtb]
            command_list += [fcpfx_mtb]
        elif sample_mdmap[sample_name] == "ecoli":
            command_list += [bt2_ecoli]
            command_list += [gtf_ecoli]
            command_list += [fcpfx_ecoli]
        command_list += [unique_pfxfor]
        #print(" ".join(command_list))
        #print(pair_idx % n_nodes)
        workload[pair_idx % n_nodes].append(command_list)
print(i)


scripts = []

# move to where the work will be done
os.chdir(save_dir)

# make a SLURM request for each piece of work
# print the files here too, just to see what 
# they look like
for work_idx in range(len(workload)):
    work = workload[work_idx]
    print(len(work))
    script_name = "process_RNAseq0_" + str(work_idx) + ".sh"
    scripts.append(script_name)
    with open(script_name, "w") as script_loc:
        print(script_name)
        script_base =  ["#!/bin/bash"]
        script_base += ["#SBATCH --partition=main"]
        script_base += ["#SBATCH --requeue"]
        script_base += ["#SBATCH --job-name=Yang_" + str(work_idx) ]
        script_base += ["#SBATCH --ntasks=1"]
        script_base += ["#SBATCH --cpus-per-task=" + str(n_cores)]
        script_base += ["#SBATCH --mem=" + str(ram)]
        script_base += ["#SBATCH --time=36:00:00"]
        script_base += ["#SBATCH --output=Yang_" + str(work_idx) + ".log.txt"]
        for line in script_base:
            print(line)
            script_loc.write(line + "\n")
        for read_pair_instructions in work:
            line = " ".join(read_pair_instructions)
            print(line)
            script_loc.write(line + "\n")
        lastline = "exit"
        print(lastline)
        script_loc.write(lastline + "\n")
        
# finally write the master script to call the scripts
with open("RNAseq0_controller.sh", "w") as caller:
    caller.write("#!/bin/bash" + "\n")
    for script_name in scripts:
        line = "sbatch " + script_name 
        print(line)
        caller.write(line + "\n")
    

72
72
12
process_RNAseq0_0.sh
#!/bin/bash
#SBATCH --partition=main
#SBATCH --requeue
#SBATCH --job-name=Yang_0
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=16000
#SBATCH --time=36:00:00
#SBATCH --output=Yang_0.log.txt
./process_reads.sh /scratch/as2654/RNAseq0/11_L002_ds.9f49b088c9f74001b79064567d253a75/IR3_S11_L002_R1_001.fastq.gz /scratch/as2654/RNAseq0/11_L002_ds.9f49b088c9f74001b79064567d253a75/IR3_S11_L002_R2_001.fastq.gz 8 16000 /mnt/scratch /scratch/as2654/RNAseq0 H37Rv_Alland /scratch/as2654/RNAseq0/H37Rv_Alland.gtf transcript IR3_S11_L002
./process_reads.sh /scratch/as2654/RNAseq0/12_L001_ds.7f8e264de73747d3894f21f9e8788339/IC1_S12_L001_R1_001.fastq.gz /scratch/as2654/RNAseq0/12_L001_ds.7f8e264de73747d3894f21f9e8788339/IC1_S12_L001_R2_001.fastq.gz 8 16000 /mnt/scratch /scratch/as2654/RNAseq0 H37Rv_Alland /scratch/as2654/RNAseq0/H37Rv_Alland.gtf transcript IC1_S12_L001
./process_reads.sh /scratch/as2654/RNAseq0/8_L002_ds.1dd835d2dd6c44dba2dd59d512876bb2/IB3_S8_L00